<a href="https://colab.research.google.com/github/danchaud-vincent/tensorflow-deep-learning/blob/main/05_Transfer_learning_in_tensorflow_part2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. Transfer Learning with TensorFlow Part 2: Fine-tuning

In the previous section, we saw how we could leverage feature extraction transfer learning to get far better results on our Food Vision project than building our own models (even with less data).

Now we're going to cover another type of transfer learning: fine-tuning.

In **fine-tuning transfer learning** the pre-trained model weights from another model are unfrozen and tweaked during to better suit your own data.

For feature extraction transfer learning, you may only train the top 1-3 layers of a pre-trained model with your own data, in fine-tuning transfer learning, you might train 1-3+ of pre-trained model.

![](https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/images/05-transfer-learning-feature-extraction-vs-fine-tuning.png)

# What we're going to cover

We're going to go through the follow with TensorFlow:
- Introduce fine-tuning, a type of transfer learning to modify a pre-trained model to be more suited to your data.
- Using the Keras Functional API (a different way to build models in Keras)
- Using a smaller dataset to experiment faster (e.g. 1-10% of training samples of 10 classes of food)
- Data augmentation (how to make your training dataset more diverse without adding more data)
- Running a series of modelling experiments on our Food Vision data:
  - Model 0: a transfer learning model using Keras Functional API
  - Model 1: a feature extraction transfer learning model on 1% of the data with data augmentation
  - Model 2 : a feature extraction transfer learning model on 10 % of the data with data augmentation
  - Model 3: a fine-tuned transfer learning model on 10% of the data
  - Model 4: a fine-tuned transfer learning model on 100% of the data
- Introduce the ModelCheckpoint callback to save intermediate training results
- Compare model experiments results using TensorBoard

In [1]:
# Using a GPU?
!nvidia-smi

Fri Sep 30 18:16:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Creating helper functions

Throughout your machine learning experiments, you'll likely come accross snippets of code you want to use over and over again.

For example, a plotting function which plots a model's `history` object (see `plot_loss_curves()` below).

You could recreate these functions over and over again. But as you might have guessed, rewritting the same functions becomes tedious.

One of the solutions is to store them in a helper functions script such as `helper_functions.py`. And then import the necessary functionality when you need it.


Let's see what this looks like.

In [2]:
# Get helper_functions.py script from course GitHub
!wget https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/utils/helper_functions.py 

--2022-09-30 18:16:24--  https://raw.githubusercontent.com/danchaud-vincent/tensorflow-deep-learning/main/utils/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2614 (2.6K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]   2.55K  --.-KB/s    in 0s      

2022-09-30 18:16:25 (51.1 MB/s) - ‘helper_functions.py’ saved [2614/2614]



In [3]:
# import Helper functions we're going to use
from helper_functions import create_tensorboard_callback, plot_loss_curves, walk_through_dir, unzip_data

### 1O Food Classes: Working with less data

We saw in the previous notebook that we could get great results with only 10% of the training data using transfer learning with TensorFlow Hub.

In this notebook, we're going to continue to work with smaller subsets of the data, except this time we'll have a look at how we can use the in-built pretrained models within the `tf.keras.applications` module as well as how to fine-tune them to our own custom dataset.

We'll also practice using a new but similar dataloader function to what we've used before, `image_dataset_from_directory()` which is part of the `tf.keras.preprocessing` module.

Finally, we'll also be practicing using the [`Keras Functional API`](https://keras.io/guides/functional_api/) for building deep learning models. The Functional API is a more flexible way to create models than the tf.keras.Sequential API.

In [4]:
# Get 10% of the data of the 10 classes
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

--2022-09-30 18:16:27--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 74.125.24.128, 142.250.4.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M  52.6MB/s    in 3.1s    

2022-09-30 18:16:31 (52.6 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
# unzip the data
unzip_data("/content/10_food_classes_10_percent.zip")

In [9]:
# walkthrough the directories
walk_through_dir("/content/10_food_classes_10_percent")

/content/10_food_classes_10_percent: There are 2 directories and 0 files
/content/10_food_classes_10_percent/train: There are 10 directories and 0 files
/content/10_food_classes_10_percent/train/steak: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/pizza: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/ramen: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/hamburger: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/chicken_wings: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/fried_rice: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/grilled_salmon: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/sushi: There are 0 directories and 75 files
/content/10_food_classes_10_percent/train/chicken_curry: There are 0 directories and 75 files
/content/10_food_classes_10_percent/tra